# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carrick-on-suir,52.3492,-7.4131,16.47,74,94,2.22,IE,05/07/2024
1,1,longyearbyen,78.2186,15.6401,0.91,75,75,1.54,SJ,05/07/2024
2,2,invercargill,-46.4000,168.3500,6.12,90,96,2.24,NZ,05/07/2024
3,3,waitangi,-43.9535,-176.5597,9.63,55,22,3.48,NZ,05/07/2024
4,4,albany,42.6001,-73.9662,19.82,50,0,0.89,US,05/07/2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "EsriImagery", 
    size = "Humidity", 
    scale = 0.7,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 29) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 5) & 
    (city_data_df['Cloudiness'] < 10) & 
    (city_data_df['Humidity'] < 80) & 
    (city_data_df['Humidity'] > 50)
]

# Drop any rows with null values
columns_to_check = ['Max Temp', 'Wind Speed', 'Cloudiness', 'Humidity']
ideal_cities_df = ideal_cities_df.dropna(subset=columns_to_check)

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,133,grand gaube,-20.0064,57.6608,25.22,75,0,4.02,MU,05/07/2024
172,172,yarmouth,41.7057,-70.2286,21.19,78,0,4.63,US,05/07/2024
194,194,tura,25.5198,90.2201,23.88,67,6,1.60,IN,05/07/2024
245,245,brisas de zicatela,15.8369,-97.0419,27.29,74,0,1.03,MX,05/07/2024
357,357,sugarcreek,41.4215,-79.8812,23.73,56,0,0.00,US,05/07/2024
379,379,prado,-17.3411,-39.2208,28.67,67,3,3.99,BR,05/07/2024
457,457,karratha,-20.7377,116.8463,25.99,68,0,2.43,AU,05/07/2024
471,471,marsh harbour,26.5412,-77.0636,24.52,67,0,4.74,BS,05/07/2024
483,483,barroso,-21.1869,-43.9758,25.10,56,0,3.48,BR,05/07/2024
562,562,toliara,-23.3500,43.6667,23.77,69,0,4.21,MG,05/07/2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
133,grand gaube,MU,-20.0064,57.6608,75,
172,yarmouth,US,41.7057,-70.2286,78,
194,tura,IN,25.5198,90.2201,67,
245,brisas de zicatela,MX,15.8369,-97.0419,74,
357,sugarcreek,US,41.4215,-79.8812,56,
379,prado,BR,-17.3411,-39.2208,67,
457,karratha,AU,-20.7377,116.8463,68,
471,marsh harbour,BS,26.5412,-77.0636,67,
483,barroso,BR,-21.1869,-43.9758,56,
562,toliara,MG,-23.3500,43.6667,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "categories":"accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request in JSON format using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grand gaube - nearest hotel: Hotel Lux- Grand Gaube
yarmouth - nearest hotel: The Inn At Cape Cod
tura - nearest hotel: Hotel Polo Orchid
brisas de zicatela - nearest hotel: Casa de Olas
sugarcreek - nearest hotel: Quality Inn & Suites
prado - nearest hotel: Pousada Recanto do Prado
karratha - nearest hotel: Karratha International Hotel
marsh harbour - nearest hotel: Lofty Fig Villas
barroso - nearest hotel: Hotel Gold
toliara - nearest hotel: Ambary
port saint john's - nearest hotel: Outback Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
133,grand gaube,MU,-20.0064,57.6608,75,Hotel Lux- Grand Gaube
172,yarmouth,US,41.7057,-70.2286,78,The Inn At Cape Cod
194,tura,IN,25.5198,90.2201,67,Hotel Polo Orchid
245,brisas de zicatela,MX,15.8369,-97.0419,74,Casa de Olas
357,sugarcreek,US,41.4215,-79.8812,56,Quality Inn & Suites
379,prado,BR,-17.3411,-39.2208,67,Pousada Recanto do Prado
457,karratha,AU,-20.7377,116.8463,68,Karratha International Hotel
471,marsh harbour,BS,26.5412,-77.0636,67,Lofty Fig Villas
483,barroso,BR,-21.1869,-43.9758,56,Hotel Gold
562,toliara,MG,-23.3500,43.6667,69,Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "EsriWorldStreetMap", 
    size = "Humidity", 
    scale = 0.7,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)